In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_7/models/B5R2b5_AN_1FC_fold7_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold7.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4600, 25)
(4600, 25)
Normal:  (3005, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4600 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.04711146, -0.08446833,  0.18320592, ..., -0.08832739,
        -0.04407514, -0.13813089],
       [-0.00815201, -0.10861675,  0.33255398, ..., -0.01943618,
         0.05616603, -0.14787361],
       [ 0.02997877, -0.10421312,  0.18676412, ..., -0.10354713,
        -0.06477267, -0.09186015],
       ...,
       [ 0.27264914,  0.5557009 , -0.1856048 , ...,  0.91319305,
         0.21213032,  0.82590908],
       [-0.16361761,  0.50222629, -0.13607937, ...,  1.01483834,
        -0.18994334,  0.91728932],
       [-0.13901442,  0.32038614, -0.14622833, ...,  0.44481358,
        -0.14334503,  0.00954154]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.047111,-0.084468,0.183206,0.106957,-0.197840,-0.146146,-0.014713,-0.039870,-0.177392,-0.150429,...,-0.090479,-0.121454,-0.161678,-0.124900,-0.054681,0.054666,-0.170043,-0.088327,-0.044075,-0.138131
1,-0.008152,-0.108617,0.332554,0.065598,-0.212385,-0.154046,0.185813,-0.174432,-0.162742,-0.105734,...,-0.051664,0.049583,-0.205449,-0.102098,0.080151,0.071277,-0.140525,-0.019436,0.056166,-0.147874
2,0.029979,-0.104213,0.186764,0.072720,-0.206438,-0.132990,0.390629,-0.209839,-0.182634,-0.188508,...,-0.079593,0.011199,-0.160709,-0.079858,0.067272,0.007184,-0.215518,-0.103547,-0.064773,-0.091860
3,0.197579,-0.117897,0.258689,0.024050,-0.209905,-0.083890,0.158171,-0.129445,-0.176343,-0.175681,...,-0.032481,-0.012497,-0.096396,-0.083368,0.002003,0.127952,-0.183598,-0.122935,0.011867,-0.153681
4,-0.001912,-0.121546,0.375175,0.073468,-0.217575,-0.134245,0.273981,-0.185224,-0.157108,-0.152178,...,-0.097518,-0.035658,-0.160212,-0.108840,-0.054425,0.013421,-0.169608,-0.113710,-0.100158,-0.153804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.491096,1.002230,-0.172730,-0.136594,0.595726,0.547629,-0.132018,0.425446,0.839854,0.207744,...,0.192134,-0.163361,0.597703,1.080752,0.385910,-0.171571,-0.167495,0.940307,-0.085193,0.750029
4596,-0.011974,0.148026,-0.153517,0.288432,0.228395,0.791473,-0.186303,0.526963,0.464647,0.790330,...,0.447034,-0.121906,0.641986,1.075142,0.025029,-0.189784,-0.117060,0.447950,-0.152273,0.707199
4597,0.272649,0.555701,-0.185605,0.503418,0.345621,1.116217,-0.118170,0.336528,0.322378,0.282140,...,0.642975,-0.124624,0.602788,1.042315,0.289419,-0.125635,-0.149157,0.913193,0.212130,0.825909
4598,-0.163618,0.502226,-0.136079,-0.055631,0.857396,0.303407,-0.183531,0.703226,0.402094,0.883147,...,0.317987,-0.142716,0.443004,0.922948,-0.117837,-0.189822,-0.126096,1.014838,-0.189943,0.917289


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.047111,-0.084468,0.183206,0.106957,-0.197840,-0.146146,-0.014713,-0.039870,-0.177392,-0.150429,...,-0.054681,0.054666,-0.170043,-0.088327,-0.044075,-0.138131,Abnormal,P31,FP-B,AB01
1,-0.008152,-0.108617,0.332554,0.065598,-0.212385,-0.154046,0.185813,-0.174432,-0.162742,-0.105734,...,0.080151,0.071277,-0.140525,-0.019436,0.056166,-0.147874,Abnormal,P1,FP-A,AB01
2,0.029979,-0.104213,0.186764,0.072720,-0.206438,-0.132990,0.390629,-0.209839,-0.182634,-0.188508,...,0.067272,0.007184,-0.215518,-0.103547,-0.064773,-0.091860,Abnormal,P2,FP-A,AB01
3,0.197579,-0.117897,0.258689,0.024050,-0.209905,-0.083890,0.158171,-0.129445,-0.176343,-0.175681,...,0.002003,0.127952,-0.183598,-0.122935,0.011867,-0.153681,Abnormal,P42,FP-B,AB01
4,-0.001912,-0.121546,0.375175,0.073468,-0.217575,-0.134245,0.273981,-0.185224,-0.157108,-0.152178,...,-0.054425,0.013421,-0.169608,-0.113710,-0.100158,-0.153804,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,0.491096,1.002230,-0.172730,-0.136594,0.595726,0.547629,-0.132018,0.425446,0.839854,0.207744,...,0.385910,-0.171571,-0.167495,0.940307,-0.085193,0.750029,Normal,P32,FP-B,Normal
4596,-0.011974,0.148026,-0.153517,0.288432,0.228395,0.791473,-0.186303,0.526963,0.464647,0.790330,...,0.025029,-0.189784,-0.117060,0.447950,-0.152273,0.707199,Normal,P42,FP-B,Normal
4597,0.272649,0.555701,-0.185605,0.503418,0.345621,1.116217,-0.118170,0.336528,0.322378,0.282140,...,0.289419,-0.125635,-0.149157,0.913193,0.212130,0.825909,Normal,P52,FP-C,Normal
4598,-0.163618,0.502226,-0.136079,-0.055631,0.857396,0.303407,-0.183531,0.703226,0.402094,0.883147,...,-0.117837,-0.189822,-0.126096,1.014838,-0.189943,0.917289,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold7_1.csv') #เปลี่ยนชื่อไฟล์